In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# 1. Data Understanding / Preparation

## 1.1 Data Retrieval

In [2]:
# Reading Accounts' Data
acc = pd.read_csv("./processed/account.csv", sep=";")

# Reading Transactions' Data
trans = pd.read_csv("./processed/trans_dev.csv", sep=";");
#trans = pd.read_csv("./processed/trans_comp.csv", sep=";");

# Reading Loans' Data
loan = pd.read_csv("./processed/loan_dev.csv", sep=";");
#loan = pd.read_csv("./processed/loan_comp.csv", sep=";");

# Reading Districts' Data
dist = pd.read_csv("./raw/district.csv", sep=";");

# Reading Disponents' Data
disp = pd.read_csv("./processed/disp.csv", sep=";");

# Reading Cards' Data
card = pd.read_csv("./processed/card_dev.csv", sep=";");
#card = pd.read_csv("./processed/card_comp.csv", sep=";");

# Reading Clients' Data
clie = pd.read_csv("./processed/client.csv", sep=";");

In [3]:
# Confirming all files were read successfully.
# General overview of number of attributes (columns) and entries (rows) per table, missing values, types of variables.
#for df in [acc, trans, loan, dist, disp, card, clie]:
#    print(df.info())
#    print("\n")

## 1.2 Data Integration - Grouping all columns of all files into a single dataframe

In [4]:
# Merging Accounts' and Transactions' Data

complete = pd.merge(acc, pd.DataFrame(trans.groupby('account_id').size(), columns=['# trans']), right_index=True, left_on='account_id', how="outer")

# Oper 1 - credit in cash, Oper 2 - credit card withdrawal, Oper 3 - collection from another bank, Oper 4 - remittance to another bank, Oper 5 - withdrawal in cash
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='credit in cash'].groupby('account_id').size(), columns=['oper #1']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='credit card withdrawal'].groupby('account_id').size(), columns=['oper #2']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='collection from another bank'].groupby('account_id').size(), columns=['oper #3']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='remittance to another bank'].groupby('account_id').size(), columns=['oper #4']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['operation']=='withdrawal in cash'].groupby('account_id').size(), columns=['oper #5']), right_index=True, left_on='account_id', how="outer")

# Type 1 - withdrawal, Type 2 - withdrawal in cash, Type 3 - credit 
complete = pd.merge(complete, pd.DataFrame(trans[trans['type']=='withdrawal'].groupby('account_id').size(), columns=['type #1']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['type']=='withdrawal in cash'].groupby('account_id').size(), columns=['type #2']), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans[trans['type']=='credit'].groupby('account_id').size(), columns=['type #3']), right_index=True, left_on='account_id', how="outer")

complete = pd.merge(complete, pd.DataFrame(trans.groupby('account_id').amount.mean().round(2)).rename(columns={'amount':'avg trans amount'}), right_index=True, left_on='account_id', how="outer")
complete = pd.merge(complete, pd.DataFrame(trans.groupby('account_id').balance.mean().round(2)).rename(columns={'balance':'avg trans balance'}), right_index=True, left_on='account_id', how="outer")
# (4500, 15)

# ------------------------

# Merging with Loans' Data
complete = pd.merge(complete, loan.rename(columns={'date': 'loan date', 'amount': 'loan amount', 'duration': 'loan duration (mths)', 'payments': 'loan payments', 'status': 'loan status'}))
# (328, 21)

# ------------------------

# Merging with Districts' Data
complete = pd.merge(complete, dist.rename(columns={'code ': 'district_id', 'name ': 'district_name', 'region': 'district_region'}))
# (328, 36)

# ------------------------

# Merging Disp' and Cards' Data
disp = disp.rename(columns={"type": "disp type"})
disp['has disponent'] = np.where(disp['disp type'] == 'DISPONENT', True, False)
card = card.rename(columns={"type": "card type"})
disp_card = pd.merge(disp, card, how="outer")
# (5369, 9)

# ------------------------

# Merging Previous with Clients' Data
disp_card_clie = pd.merge(disp_card, clie).drop_duplicates(subset=['account_id']).drop(columns=['has disponent', 'disp type', 'district_id', 'disp_id'])
# (4500, 7)
temp = pd.merge(disp_card_clie, pd.DataFrame(disp_card.groupby('account_id').size(), columns = ['count owner']), right_index=True, left_on='account_id', how="outer")
disp_card_clie['has disponent'] = np.where(temp['count owner'] == 2, True, False)
# (4500, 8)

# ------------------------

# Merging Previous subset with the rest
complete = pd.merge(complete, disp_card_clie)


print(complete.shape)
complete


(328, 43)


,account_id,district_id,frequency,date,# trans,oper #1,oper #2,oper #3,oper #4,oper #5,type #1,type #2,type #3,avg trans amount,avg trans balance,loan_id,loan date,loan amount,loan duration (mths),loan payments,loan status,district_name,district_region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,client_id,card_id,card type,issued,birthdate,sex,has disponent
0,5270,44,monthly issuance,1993-01-13,53.0,1.0,NaN,11.0,NaN,31.0,28.0,3.0,22.0,17600.36,69954.57,6077,1993-11-22,79608,24,3317,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,6367,NaN,NaN,NaN,1970-04-28,M,False
1,5385,44,weekly issuance,1993-05-21,56.0,3.0,NaN,10.0,6.0,27.0,32.0,1.0,23.0,5778.79,29069.12,6103,1994-04-07,149340,60,2489,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,6499,NaN,NaN,NaN,1963-02-01,M,False
2,8027,44,weekly issuance,1993-07-07,60.0,2.0,NaN,10.0,NaN,38.0,33.0,5.0,22.0,13255.80,55570.30,6642,1994-06-05,77640,12,6470,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,9915,NaN,NaN,NaN,1967-07-10,M,False
3,3037,44,monthly issuance,1994-05-26,117.0,26.0,NaN,NaN,NaN,71.0,70.0,1.0,46.0,10720.31,46487.00,5593,1996-04-18,277884,36,7719,-1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,3670,NaN,NaN,NaN,1980-09-17,F,False
4,10942,44,weekly issuance,1994-06-05,130.0,40.0,NaN,NaN,12.0,62.0,72.0,2.0,56.0,15168.36,63527.42,7227,1995-10-02,314520,60,5242,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,13451,NaN,NaN,NaN,1955-11-18,F,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,4462,73,weekly issuance,1995-12-27,10.0,4.0,NaN,NaN,NaN,3.0,3.0,NaN,7.0,13609.12,34798.28,5892,1996-05-14,66480,24,2770,-1,Opava,north Moravia,182027,17,49,12,2,7,56.4,8746,3.33,3.74,90,4355,4433,5384,NaN,NaN,NaN,1935-07-21,M,False
324,5001,53,monthly issuance,1995-12-15,82.0,27.0,NaN,NaN,6.0,39.0,41.0,4.0,37.0,17234.72,76232.70,6015,1996-11-09,110112,24,4588,1,Blansko,south Moravia,107911,88,33,6,2,7,50.9,8240,2.53,3.56,99,1850,1903,6042,NaN,NaN,NaN,1963-10-12,M,False
325,3084,59,weekly issuance,1996-01-28,29.0,8.0,NaN,NaN,NaN,15.0,13.0,2.0,14.0,11379.26,37095.84,5606,1996-08-28,253512,36,7042,-1,Kromeriz,south Moravia,108871,41,29,7,2,6,62.1,8444,3.24,3.47,106,2595,2305,3730,NaN,NaN,NaN,1945-08-07,F,False
326,11054,59,weekly issuance,1996-02-01,31.0,3.0,NaN,7.0,3.0,12.0,14.0,1.0,16.0,7832.08,32435.32,7246,1996-08-20,148920,60,2482,1,Kromeriz,south Moravia,108871,41,29,7,2,6,62.1,8444,3.24,3.47,106,2595,2305,13590,NaN,NaN,NaN,1977-08-03,M,False


### 1.2.1 Replacing by 0 columns refering to number of transactions (operations / types) that are NaN (weren't filled in

In [5]:
# NaN value is present in # operation / # type columns and represents that 0 were found. Therefore 0 is a more suitable value.
for col in ['oper #1', 'oper #2', 'oper #3', 'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3']:
    complete[col] = complete[col].fillna(0)
    
complete.head()

,account_id,district_id,frequency,date,# trans,oper #1,oper #2,oper #3,oper #4,oper #5,type #1,type #2,type #3,avg trans amount,avg trans balance,loan_id,loan date,loan amount,loan duration (mths),loan payments,loan status,district_name,district_region,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,client_id,card_id,card type,issued,birthdate,sex,has disponent
0,5270,44,monthly issuance,1993-01-13,53.0,1.0,0.0,11.0,0.0,31.0,28.0,3.0,22.0,17600.36,69954.57,6077,1993-11-22,79608,24,3317,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,6367,NaN,NaN,NaN,1970-04-28,M,False
1,5385,44,weekly issuance,1993-05-21,56.0,3.0,0.0,10.0,6.0,27.0,32.0,1.0,23.0,5778.79,29069.12,6103,1994-04-07,149340,60,2489,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,6499,NaN,NaN,NaN,1963-02-01,M,False
2,8027,44,weekly issuance,1993-07-07,60.0,2.0,0.0,10.0,0.0,38.0,33.0,5.0,22.0,13255.80,55570.30,6642,1994-06-05,77640,12,6470,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,9915,NaN,NaN,NaN,1967-07-10,M,False
3,3037,44,monthly issuance,1994-05-26,117.0,26.0,0.0,0.0,0.0,71.0,70.0,1.0,46.0,10720.31,46487.00,5593,1996-04-18,277884,36,7719,-1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,3670,NaN,NaN,NaN,1980-09-17,F,False
4,10942,44,weekly issuance,1994-06-05,130.0,40.0,0.0,0.0,12.0,62.0,72.0,2.0,56.0,15168.36,63527.42,7227,1995-10-02,314520,60,5242,1,Chrudim,east Bohemia,105606,77,26,7,2,7,53.0,8254,2.79,3.76,97,2166,2325,13451,NaN,NaN,NaN,1955-11-18,F,False


### 1.2.2 Deleting columns whose only purpose was to help in the join of tables (IDs) and/or are redundant

In [6]:
# Removing attributes. district_name / district_region are redundant because they can be derived by the rest of the district columns
complete = complete.drop(columns=['account_id', 'district_id', 'district_name', 'district_region', 'client_id', 'card_id'])

# Doesn't seem to be useful, especially considering the very limited amount of data related to cards
complete = complete.drop(columns=['issued'])

print(complete.shape)
complete.columns

(328, 36)


Index(['frequency', 'date', '# trans', 'oper #1', 'oper #2', 'oper #3',
       'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3',
       'avg trans amount', 'avg trans balance', 'loan_id', 'loan date',
       'loan amount', 'loan duration (mths)', 'loan payments', 'loan status',
       'no. of inhabitants', 'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'unemploymant rate '95 ', 'unemploymant rate '96 ',
       'no. of enterpreneurs per 1000 inhabitants ',
       'no. of commited crimes '95 ', 'no. of commited crimes '96 ',
       'card type', 'birthdate', 'sex', 'has disponent'],
      dtype='object')

In [7]:
complete = complete.rename(columns={'date': 'acc date'})
complete.columns

Index(['frequency', 'acc date', '# trans', 'oper #1', 'oper #2', 'oper #3',
       'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3',
       'avg trans amount', 'avg trans balance', 'loan_id', 'loan date',
       'loan amount', 'loan duration (mths)', 'loan payments', 'loan status',
       'no. of inhabitants', 'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'unemploymant rate '95 ', 'unemploymant rate '96 ',
       'no. of enterpreneurs per 1000 inhabitants ',
       'no. of commited crimes '95 ', 'no. of commited crimes '96 ',
       'card type', 'birthdate', 'sex', 'has disponent'],
      dtype='object')

### 1.2.3 Account id as index

In [8]:
complete = complete.set_index('loan_id')

In [9]:
complete['age_at_loan'] = (pd.to_datetime(complete['loan date']) - pd.to_datetime(complete['birthdate'])).dt.total_seconds() / (365.25*24*3600)
complete['acc_age_at_loan'] = (pd.to_datetime(complete['loan date']) - pd.to_datetime(complete['acc date'])).dt.total_seconds() / (365.25*24*3600)

In [10]:
complete.loc[complete["unemploymant rate '95 "] == '?', "unemploymant rate '95 "] = complete["unemploymant rate '96 "]
complete.loc[complete["unemploymant rate '96 "] == '?', "unemploymant rate '96 "] = complete["unemploymant rate '95 "]

In [11]:
complete["unemploymant rate"] = (complete["unemploymant rate '95 "].astype('float') + complete["unemploymant rate '96 "].astype('float')) / 2.0
complete = complete.drop(columns=["unemploymant rate '95 ", "unemploymant rate '96 "])

In [12]:
complete.loc[complete["no. of commited crimes '95 "] == '?', "no. of commited crimes '95 "] = complete["no. of commited crimes '96 "]
complete.loc[complete["no. of commited crimes '96 "] == '?', "no. of commited crimes '96 "] = complete["no. of commited crimes '95 "]

In [13]:
complete["no. of commited crimes per capita"] = (complete["no. of commited crimes '95 "].astype('float') + complete["no. of commited crimes '96 "].astype('float')) / 2.0 / complete["no. of inhabitants"]
complete = complete.drop(columns=["no. of commited crimes '95 ", "no. of commited crimes '96 "])

In [14]:
complete['card type'] = complete['card type'].fillna('None')

In [15]:
#complete.loc[complete['age at loan (Years)'] < 18]

In [16]:
complete.columns

Index(['frequency', 'acc date', '# trans', 'oper #1', 'oper #2', 'oper #3',
       'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3',
       'avg trans amount', 'avg trans balance', 'loan date', 'loan amount',
       'loan duration (mths)', 'loan payments', 'loan status',
       'no. of inhabitants', 'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'no. of enterpreneurs per 1000 inhabitants ', 'card type', 'birthdate',
       'sex', 'has disponent', 'age_at_loan', 'acc_age_at_loan',
       'unemploymant rate', 'no. of commited crimes per capita'],
      dtype='object')

In [17]:
complete = complete[['loan status', 'frequency', 'acc date', '# trans', 'oper #1', 'oper #2', 'oper #3',
       'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3',
       'avg trans amount', 'avg trans balance', 'loan date', 'loan amount',
       'loan duration (mths)', 'loan payments',
       'no. of inhabitants', 'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'no. of enterpreneurs per 1000 inhabitants ', 'card type', 'birthdate',
       'sex', 'has disponent', 'age_at_loan', 'acc_age_at_loan',
       'unemploymant rate', 'no. of commited crimes per capita']]
complete.shape

(328, 35)

In [18]:
complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 6077 to 6751
Data columns (total 35 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   loan status                                        328 non-null    int64  
 1   frequency                                          328 non-null    object 
 2   acc date                                           328 non-null    object 
 3   # trans                                            328 non-null    float64
 4   oper #1                                            328 non-null    float64
 5   oper #2                                            328 non-null    float64
 6   oper #3                                            328 non-null    float64
 7   oper #4                                            328 non-null    float64
 8   oper #5                                            328 non-null    float64
 9   type #

In [19]:
complete.columns

Index(['loan status', 'frequency', 'acc date', '# trans', 'oper #1', 'oper #2',
       'oper #3', 'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3',
       'avg trans amount', 'avg trans balance', 'loan date', 'loan amount',
       'loan duration (mths)', 'loan payments', 'no. of inhabitants',
       'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'no. of enterpreneurs per 1000 inhabitants ', 'card type', 'birthdate',
       'sex', 'has disponent', 'age_at_loan', 'acc_age_at_loan',
       'unemploymant rate', 'no. of commited crimes per capita'],
      dtype='object')

In [20]:
for column in ['acc date', 'loan date', 'birthdate']:
    complete[column] = pd.to_datetime(complete[column])

In [21]:
def ordinal_encode(df, column, ordering):
    df = df.copy()
    df[column] = df[column].apply(lambda x : ordering.index(x))
    return df

def special_binary_encode(df, column, negative_value):
    df = df.copy()
    df['card_owner'] = df[column].apply(lambda x : 0 if x == negative_value else 1)
    # df = df.drop(column, axis=1)
    return df

In [22]:
#complete['account_year'] = complete['acc date'].apply(lambda x : x.year)
#complete['account_month'] = complete['acc date'].apply(lambda x : x.month)
#complete['account_day'] = complete['acc date'].apply(lambda x : x.day)

#complete['loan_year'] = complete['loan date'].apply(lambda x : x.year)
#complete['loan_month'] = complete['loan date'].apply(lambda x : x.month)
#complete['loan_day'] = complete['loan date'].apply(lambda x : x.day)

#complete['birth_year'] = complete['birthdate'].apply(lambda x : x.year)
#complete['birth_month'] = complete['birthdate'].apply(lambda x : x.month)
#complete['birth_day'] = complete['birthdate'].apply(lambda x : x.day)

#complete = complete.drop(['acc date', 'loan date', 'birthdate'], axis=1)

In [23]:
#freq_ordering = [
#    'monthly issuance',
#    'weekly issuance',
#    'issuance after transaction'
#]
#complete = ordinal_encode(complete, 'frequency', ordering=freq_ordering)

complete = pd.get_dummies(complete, columns=['frequency'])

complete = special_binary_encode(complete, 'card type', negative_value='None')

complete = pd.get_dummies(complete, columns=['card type'])

## 1.3 Complete ('joined') data

In [24]:
complete

,loan status,acc date,# trans,oper #1,oper #2,oper #3,oper #4,oper #5,type #1,type #2,type #3,avg trans amount,avg trans balance,loan date,loan amount,loan duration (mths),loan payments,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,no. of enterpreneurs per 1000 inhabitants,birthdate,sex,has disponent,age_at_loan,acc_age_at_loan,unemploymant rate,no. of commited crimes per capita,frequency_issuance after transaction,frequency_monthly issuance,frequency_weekly issuance,card_owner,card type_None,card type_classic,card type_gold,card type_junior
loan_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6077,1,1993-01-13,53.0,1.0,0.0,11.0,0.0,31.0,28.0,3.0,22.0,17600.36,69954.57,1993-11-22,79608,24,3317,105606,77,26,7,2,7,53.0,8254,97,1970-04-28,M,False,23.570157,0.856947,3.275,0.021263,0,1,0,0,1,0,0,0
6103,1,1993-05-21,56.0,3.0,0.0,10.0,6.0,27.0,32.0,1.0,23.0,5778.79,29069.12,1994-04-07,149340,60,2489,105606,77,26,7,2,7,53.0,8254,97,1963-02-01,M,False,31.178645,0.878850,3.275,0.021263,0,0,1,0,1,0,0,0
6642,1,1993-07-07,60.0,2.0,0.0,10.0,0.0,38.0,33.0,5.0,22.0,13255.80,55570.30,1994-06-05,77640,12,6470,105606,77,26,7,2,7,53.0,8254,97,1967-07-10,M,False,26.904860,0.911704,3.275,0.021263,0,0,1,0,1,0,0,0
5593,-1,1994-05-26,117.0,26.0,0.0,0.0,0.0,71.0,70.0,1.0,46.0,10720.31,46487.00,1996-04-18,277884,36,7719,105606,77,26,7,2,7,53.0,8254,97,1980-09-17,F,False,15.583847,1.897331,3.275,0.021263,0,1,0,0,1,0,0,0
7227,1,1994-06-05,130.0,40.0,0.0,0.0,12.0,62.0,72.0,2.0,56.0,15168.36,63527.42,1995-10-02,314520,60,5242,105606,77,26,7,2,7,53.0,8254,97,1955-11-18,F,False,39.871321,1.325120,3.275,0.021263,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,-1,1995-12-27,10.0,4.0,0.0,0.0,0.0,3.0,3.0,0.0,7.0,13609.12,34798.28,1996-05-14,66480,24,2770,182027,17,49,12,2,7,56.4,8746,90,1935-07-21,M,False,60.815880,0.380561,3.535,0.024139,0,0,1,0,1,0,0,0
6015,1,1995-12-15,82.0,27.0,0.0,0.0,6.0,39.0,41.0,4.0,37.0,17234.72,76232.70,1996-11-09,110112,24,4588,107911,88,33,6,2,7,50.9,8240,99,1963-10-12,M,False,33.078713,0.903491,3.045,0.017389,0,1,0,0,1,0,0,0
5606,-1,1996-01-28,29.0,8.0,0.0,0.0,0.0,15.0,13.0,2.0,14.0,11379.26,37095.84,1996-08-28,253512,36,7042,108871,41,29,7,2,6,62.1,8444,106,1945-08-07,F,False,51.058179,0.583162,3.355,0.022504,0,0,1,0,1,0,0,0


### 1.3.1 Exporting to csv

In [25]:
complete.to_csv("./last_dev_complete_data_2.csv", sep=";")
#complete.to_csv("./last_comp_complete_data.csv", sep=";")

## 1.4 Data Analysis

### 1.4.1 Frequency - Nominal Categorical Attribute

In [26]:
complete['frequency'].describe()

KeyError: 'frequency'

In [ ]:
complete['frequency'].value_counts().plot(kind='bar')

In [ ]:
fig, ax = plt.subplots()
ax.pie(complete['frequency'].value_counts(), labels=complete['frequency'].unique(), autopct='%1.1f%%')
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Frequency types')

plt.show()

#### Remarks:
Of the 328 rows of complete data available, the majority corresponds to accounts where the frequency attribute value is 'monthly issuance'. 'monthly issuance' has 5 times more entries than 'weekly issuance' and more than 14 times more entries than 'issuance after transaction'.

This might mean that this attribute will not be of great help to predict the outcome of a loan due to the small sample of the other 2 possibilities.

### 1.4.2 Account Creation Date - Date Attribute

In [ ]:
complete['date'].describe()

In [ ]:
print("Min: ", end="");                 print(complete['date'].min())
print("Max: ", end="");                 print(complete['date'].max())

In [ ]:
complete['date'].value_counts()[:50].plot(kind='bar', figsize=(15,3))

#### Remarks:
The accounts of this dataset were all created between the years 1993 and 1996.

The date of account creation a priori doesn't seem to be related at all with the possible outcome of a loan.

### 1.4.3 Number of transactions - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['# trans'], [75 ,25])

print("Min: ", end="");                 print(complete['# trans'].min())
print("Max: ", end="");                 print(complete['# trans'].max())
print("Mean: ", end="");                print(complete['# trans'].mean())
print("Median: ", end="");              print(complete['# trans'].median())
print("Standard deviation: ", end="");  print(complete['# trans'].std())
print("Skewness: ", end="");            print(complete['# trans'].skew())  
print("Variance: ", end="");            print(complete['# trans'].var())
print("Kurtosis: ", end="");            print(complete['# trans'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['# trans'])

In [ ]:
complete['# trans'].plot.box();

#### Remarks:
The number of transactions performed by an account has a very wide range of values and every single one is at minimum 4 and at maximum 196 which justifies the high variance / standard deviation. The majority of the values is concentrated in the range \[39,103\].
By observing the skewness we can classify the data as slightly skewed (between -0.5 and 0.5 would be considered approximately symmetric). The small (in absolute value) kurtosis (it is in the range \[-1,1\]) informs that the distribution is neither too flat or too peeked.

### 1.4.4 Transaction Operations - Numerical Attributes

In [ ]:
subset = complete[['oper #1', 'oper #2', 'oper #3', 'oper #4', 'oper #5']].mean()
print(subset)
subset.plot(kind='bar', title="Average number of transactions per Operation")

#### Remarks:
The data provided is very unbalanced when it comes to the transaction operations' performed for each account of which there is information about a loan. Of the 5 options identified, 'oper \#5' ('withdrawal in cash') is by far the most common while 'oper \#2' ('credit card withdrawal') is very rare.

Therefore, this attribute might not be of much use to predict the outcome of a loan due to sample size discrepancy of the 5 possible values.

### 1.4.5 Transaction Types - Numerical Attributes

In [ ]:
subset = complete[['type #1', 'type #2', 'type #3']].mean()
subset.plot(kind='bar', title="Average number of transactions per Type")

#### Remarks: 
Similar to columns 'oper X' with the difference that in this case only one value is much less represented than the others
Therefore, similarly to the previous attribute, this one might also not be of much use to predict the outcome of a loan due to sample size of 1 of the possible values.

### 1.4.6 Average Transaction Amount - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['avg trans amount'], [75 ,25])

print("Min: ", end="");                 print(complete['avg trans amount'].min())
print("Max: ", end="");                 print(complete['avg trans amount'].max())
print("Mean: ", end="");                print(complete['avg trans amount'].mean())
print("Median: ", end="");              print(complete['avg trans amount'].median())
print("Standard deviation: ", end="");  print(complete['avg trans amount'].std())
print("Skewness: ", end="");            print(complete['avg trans amount'].skew())  
print("Variance: ", end="");            print(complete['avg trans amount'].var())
print("Kurtosis: ", end="");            print(complete['avg trans amount'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

# var() method returns the unbiased variance
# Medium-High std which indicates that the data is spread out and not clustered around the mean
# Skewness > 1 => it is skewed to the right

In [ ]:
sns.distplot(complete['avg trans amount'])

In [ ]:
complete['avg trans amount'].plot.box();

#### Remarks:
The 'average transaction amount' atribute seems to follow approximately a normal distribution based on its graphical representation and skewness/kurtosis values.

Despite this, it is possible to identify some values outside the general landscape that can either correspond to outliers or to a real group that is not well represented in the provided data.

### 1.4.7 Average Transaction Balance - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['avg trans balance'], [75 ,25])

print("Min: ", end="");                 print(complete['avg trans balance'].min())
print("Max: ", end="");                 print(complete['avg trans balance'].max())
print("Mean: ", end="");                print(complete['avg trans balance'].mean())
print("Median: ", end="");              print(complete['avg trans balance'].median())
print("Standard deviation: ", end="");  print(complete['avg trans balance'].std())
print("Skewness: ", end="");            print(complete['avg trans balance'].skew())  
print("Variance: ", end="");            print(complete['avg trans balance'].var())
print("Kurtosis: ", end="");            print(complete['avg trans balance'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

# var() method returns the unbiased variance
# Medium-High std which indicates that the data is spread out and not clustered around the mean
# Skewness > 1 => it is skewed to the right

In [ ]:
sns.distplot(complete['avg trans balance'])

In [ ]:
complete['avg trans balance'].plot.box();

#### Remarks:
Like the previous atribute, the 'average transaction balance' seems to follow approximately a normal distribution based on its graphical representation and skewness/kurtosis values. Contrasting with the last atribute, there seem to be no possible outliers, constituting possibly a good candidate variable to predict the outcome of a loan.

### 1.4.8 Loan Date - Date Attribute

In [ ]:
complete['loan date'].describe()

In [ ]:
print("Min: ", end="");                 print(complete['loan date'].min())
print("Max: ", end="");                 print(complete['loan date'].max())

In [ ]:
complete['loan date'].value_counts()[:50].plot(kind='bar', figsize=(15,3))

#### Remarks:
The loan of this dataset all date between the years 1993 and 1996, similarly to the account creation dates.

The date of a loan, a priori, doesn't seem to be related at all with the possible outcome of a loan.

In [ ]:
q3, q1 = np.percentile(complete['age at loan (Years)'], [75 ,25])

print("Min: ", end="");                 print(complete['age at loan (Years)'].min())
print("Max: ", end="");                 print(complete['age at loan (Years)'].max())
print("Mean: ", end="");                print(complete['age at loan (Years)'].mean())
print("Median: ", end="");              print(complete['age at loan (Years)'].median())
print("Standard deviation: ", end="");  print(complete['age at loan (Years)'].std())
print("Skewness: ", end="");            print(complete['age at loan (Years)'].skew())  
print("Variance: ", end="");            print(complete['age at loan (Years)'].var())
print("Kurtosis: ", end="");            print(complete['age at loan (Years)'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['age at loan (Years)'])

In [ ]:
complete['age at loan (Years)'].plot.box()

#### Remarks:
The range of ages of the clients at the time at which the loan was provided is quite comprehensive.
The skewness indicates a distribution of the data that isn't skewed and the kurtosis points to its flatness, as it is possible to confirm by observation of the distplot.

Despite this aparent 'normality' and, by extent, possibly usefulness of this atribute it is also possible to detect that some values and under the minimum adult age (18 years) which would make these entries invalid and, therefore, creating the necessity to remove them before further analysis.

### 1.4.9 Loan Amount - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['loan amount'], [75 ,25])

print("Min: ", end="");                 print(complete['loan amount'].min())
print("Max: ", end="");                 print(complete['loan amount'].max())
print("Mean: ", end="");                print(complete['loan amount'].mean())
print("Median: ", end="");              print(complete['loan amount'].median())
print("Standard deviation: ", end="");  print(complete['loan amount'].std())
print("Skewness: ", end="");            print(complete['loan amount'].skew())  
print("Variance: ", end="");            print(complete['loan amount'].var())
print("Kurtosis: ", end="");            print(complete['loan amount'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

# Very High std which indicates that the data is spread out and not clustered around the mean
# > 1 therefore it is skewed to the right

In [ ]:
sns.distplot(complete['loan amount'])

In [ ]:
complete['loan amount'].plot.box();

#### Remarks:
The loan amount column data is made of very different values and it is skewed to the right which is confirmed by the skewness. The kurtosis can be confirmed by observation of the seaborn distplot that shows a peak around the value 100000.

It is possible to identify some values outside the general landscape that can either correspond to outliers or to a real group that is not well represented in the provided data. Due to the number of these values we assume the second hypotesis.

### 1.4.10 Loan Duration (in months) - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['loan duration (mths)'], [75 ,25])

print("Min: ", end="");                 print(complete['loan duration (mths)'].min())
print("Max: ", end="");                 print(complete['loan duration (mths)'].max())
print("Mean: ", end="");                print(complete['loan duration (mths)'].mean())
print("Median: ", end="");              print(complete['loan duration (mths)'].median())
print("Standard deviation: ", end="");  print(complete['loan duration (mths)'].std())
print("Skewness: ", end="");            print(complete['loan duration (mths)'].skew())  
print("Variance: ", end="");            print(complete['loan duration (mths)'].var())
print("Kurtosis: ", end="");            print(complete['loan duration (mths)'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
complete['loan duration (mths)'].value_counts().sort_index().plot(kind='bar')

#### Remarks:
In general, the loan duration data seems rather uniform visually, which is in agreement with the statistical analysis results: small absolute value of skewness points to a approximately normal distribution and the lower than -1 kurtosis represents a flat distribution.

Due to the very small variation in frequency of each of the values of this loan duration column, it is plausible to think it won't provide much information regarding the prediction of the outcome of a loan.

### 1.4.11 Loan Payments - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['loan payments'], [75 ,25])

print("Min: ", end="");                 print(complete['loan payments'].min())
print("Max: ", end="");                 print(complete['loan payments'].max())
print("Mean: ", end="");                print(complete['loan payments'].mean())
print("Median: ", end="");              print(complete['loan payments'].median())
print("Standard deviation: ", end="");  print(complete['loan payments'].std())
print("Skewness: ", end="");            print(complete['loan payments'].skew())  
print("Variance: ", end="");            print(complete['loan payments'].var())
print("Kurtosis: ", end="");            print(complete['loan payments'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['loan payments'])

In [ ]:
complete['loan payments'].plot.box();

#### Remarks:
Similarly to some of the previous atributes, loan payments follows approximately a normal distribution.
Besides, in some way seems to be in agreement with the loan amount atribute, as expected.

### 1.4.12 Loan Status - Nominal(?) Attribute

In [ ]:
#complete['loan status'].describe()

In [ ]:
# -1 -> not paid; 1 -> paid
#complete['loan status'].value_counts().plot(kind='bar')

#### Remarks:
By observation alone it is clear that the amount of loans where an account fulfills its commitments to the bank severely surpasses the number of entries where default credit it the result. Then, oversampling/undersampling methods might useful.

### 1.4.13 Number of district inhabitants - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['no. of inhabitants'], [75 ,25])

print("Min: ", end="");                 print(complete['no. of inhabitants'].min())
print("Max: ", end="");                 print(complete['no. of inhabitants'].max())
print("Mean: ", end="");                print(complete['no. of inhabitants'].mean())
print("Median: ", end="");              print(complete['no. of inhabitants'].median())
print("Standard deviation: ", end="");  print(complete['no. of inhabitants'].std())
print("Skewness: ", end="");            print(complete['no. of inhabitants'].skew())  
print("Variance: ", end="");            print(complete['no. of inhabitants'].var())
print("Kurtosis: ", end="");            print(complete['no. of inhabitants'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['no. of inhabitants'])

In [ ]:
complete['no. of inhabitants'].plot.box();

#### Remarks:
Without the data past 750 thousand inhabitants, the atribute would assume almost a perfect normal distribution. As is, the data distribution is substantially skewed and the kurtosis confirms the peaked characterization.
Due to the percentage of information that fits in the supposed 'outlier zone', these entries can't be considered outliers. They may just represent a densely populated area where the average corresponds to semi-urban or even rural regions and the provided data simply doesn't have information regarding regions whose population numbers fit the \[400000, 1100000\] range.

### 1.4.14 Inahbitants per municipality - Numerical Attribute

In [ ]:
fig, ax = plt.subplots(2,2)
sns.distplot(complete['no. of municipalities with inhabitants < 499 '], ax=ax[0][0])
sns.distplot(complete['no. of municipalities with inhabitants 500-1999'], ax=ax[0][1])

sns.distplot(complete['no. of municipalities with inhabitants 2000-9999 '], ax=ax[1][0])
sns.distplot(complete['no. of municipalities with inhabitants >10000 '], ax=ax[1][1])
fig.tight_layout()
fig.show()

In [ ]:
complete[['no. of municipalities with inhabitants < 499 ', 'no. of municipalities with inhabitants 500-1999', 'no. of municipalities with inhabitants 2000-9999 ', 'no. of municipalities with inhabitants >10000 ']].plot(kind='box', subplots=True, legend=False, layout=(2, 2))
plt.tight_layout()

#### Remarks:
The distributions of the number of municipalities seem very disperse and with aparent low value by themselves for loan predictions. Nonetheless, it can be especulated that many of the accounts belong to people that live in a far more densely populated area that the rest, hyposthesis that is corroborated by the previous and next data regarding the districts.

### 1.4.15 Number of cities of district - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['no. of cities '], [75 ,25])

print("Min: ", end="");                 print(complete['no. of cities '].min())
print("Max: ", end="");                 print(complete['no. of cities '].max())
print("Mean: ", end="");                print(complete['no. of cities '].mean())
print("Median: ", end="");              print(complete['no. of cities '].median())
print("Standard deviation: ", end="");  print(complete['no. of cities '].std())
print("Skewness: ", end="");            print(complete['no. of cities '].skew())  
print("Variance: ", end="");            print(complete['no. of cities '].var())
print("Kurtosis: ", end="");            print(complete['no. of cities '].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
complete['no. of cities '].value_counts().sort_index().plot(kind='bar')

In [ ]:
sns.distplot(complete['no. of cities '].value_counts())

In [ ]:
complete['no. of cities '].plot.box();

#### Remarks:
Aproximately a normal distribution.
This data doesn't seem to indicate any anomaly that might skew the outcome of a loan.

### 1.4.16 Ratio of Urban Inhabitants - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['ratio of urban inhabitants '], [75 ,25])

print("Min: ", end="");                 print(complete['ratio of urban inhabitants '].min())
print("Max: ", end="");                 print(complete['ratio of urban inhabitants '].max())
print("Mean: ", end="");                print(complete['ratio of urban inhabitants '].mean())
print("Median: ", end="");              print(complete['ratio of urban inhabitants '].median())
print("Standard deviation: ", end="");  print(complete['ratio of urban inhabitants '].std())
print("Skewness: ", end="");            print(complete['ratio of urban inhabitants '].skew())  
print("Variance: ", end="");            print(complete['ratio of urban inhabitants '].var())
print("Kurtosis: ", end="");            print(complete['ratio of urban inhabitants '].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['ratio of urban inhabitants '])

In [ ]:
complete['ratio of urban inhabitants '].plot.box();

#### Remarks:
The range of districts' ratio of urban inhabitants is very wide. The provided information regarding people that took loans points to 2 concentrations: in districts where the urban inhabitants constitute either 50 or 100 percent of the population. Due to these 2 spikes, the skewness and kurtosis conclude an approximate normal and flat distribution of the data.

### 1.4.17 Average Salary - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['average salary '], [75 ,25])

print("Min: ", end="");                 print(complete['average salary '].min())
print("Max: ", end="");                 print(complete['average salary '].max())
print("Mean: ", end="");                print(complete['average salary '].mean())
print("Median: ", end="");              print(complete['average salary '].median())
print("Standard deviation: ", end="");  print(complete['average salary '].std())
print("Skewness: ", end="");            print(complete['average salary '].skew())  
print("Variance: ", end="");            print(complete['average salary '].var())
print("Kurtosis: ", end="");            print(complete['average salary '].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['average salary '])

In [ ]:
complete['average salary '].plot.box();

#### Remarks:
The average salary atribute seems to follow graphically a distribution somewhat similar to the ratio or urban inhabitants. The second 'spike' in this column could be taken as outlier because it is separated from the rest, which is not an option because of the percentage of the entries that it represents.

### 1.4.18 Unemploymant Rate - Numerical Attribute

In [ ]:
#complete.loc[complete["unemploymant rate '95 "] == '?', "unemploymant rate '95 "] = complete["unemploymant rate '96 "]
#complete.loc[complete["unemploymant rate '96 "] == '?', "unemploymant rate '96 "] = complete["unemploymant rate '95 "]

#### Obtain the average of 95' and 96' columns

In [ ]:
#complete["unemploymant rate"] = (complete["unemploymant rate '95 "].astype('float') + complete["unemploymant rate '96 "].astype('float')) / 2.0
#complete = complete.drop(columns=["unemploymant rate '95 ", "unemploymant rate '96 "])
# complete

In [ ]:
q3, q1 = np.percentile(complete['unemploymant rate'], [75 ,25])

print("Min: ", end="");                 print(complete['unemploymant rate'].min())
print("Max: ", end="");                 print(complete['unemploymant rate'].max())
print("Mean: ", end="");                print(complete['unemploymant rate'].mean())
print("Median: ", end="");              print(complete['unemploymant rate'].median())
print("Standard deviation: ", end="");  print(complete['unemploymant rate'].std())
print("Skewness: ", end="");            print(complete['unemploymant rate'].skew())  
print("Variance: ", end="");            print(complete['unemploymant rate'].var())
print("Kurtosis: ", end="");            print(complete['unemploymant rate'].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['unemploymant rate'])

In [ ]:
complete['unemploymant rate'].plot.box();

#### Remarks:
The unemploymant rate seems to follow a normal distribution and doesn't have any outliers.

### 1.4.19 Number of enterpreneurs per 1000 inhabitants - Numerical Attribute

In [ ]:
q3, q1 = np.percentile(complete['no. of enterpreneurs per 1000 inhabitants '], [75 ,25])

print("Min: ", end="");                 print(complete['no. of enterpreneurs per 1000 inhabitants '].min())
print("Max: ", end="");                 print(complete['no. of enterpreneurs per 1000 inhabitants '].max())
print("Mean: ", end="");                print(complete['no. of enterpreneurs per 1000 inhabitants '].mean())
print("Median: ", end="");              print(complete['no. of enterpreneurs per 1000 inhabitants '].median())
print("Standard deviation: ", end="");  print(complete['no. of enterpreneurs per 1000 inhabitants '].std())
print("Skewness: ", end="");            print(complete['no. of enterpreneurs per 1000 inhabitants '].skew())  
print("Variance: ", end="");            print(complete['no. of enterpreneurs per 1000 inhabitants '].var())
print("Kurtosis: ", end="");            print(complete['no. of enterpreneurs per 1000 inhabitants '].kurtosis())
print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

In [ ]:
sns.distplot(complete['no. of enterpreneurs per 1000 inhabitants '])

In [ ]:
complete['no. of enterpreneurs per 1000 inhabitants '].plot.box();

#### Remarks:
Interestingly, the number of enterpreneurs per 1000 inhabitants follows a distribution practically identical to the ratio of urban inhabitants and average salary, which may represent that some of the columns regarding district data might be redundant.

### 1.4.20 Number of commited crimes per capita - Numerical Attribute

In [ ]:
#complete.loc[complete["no. of commited crimes '95 "] == '?', "no. of commited crimes '95 "] = complete["no. of commited crimes '96 "]
#complete.loc[complete["no. of commited crimes '96 "] == '?', "no. of commited crimes '96 "] = complete["no. of commited crimes '95 "]

#### Obtain the average of 95' and 96' columns and then divide by the number of inhabitants

In [ ]:
#complete["no. of commited crimes per capita"] = (complete["no. of commited crimes '95 "].astype('float') + complete["no. of commited crimes '96 "].astype('float')) / 2.0 / complete["no. of inhabitants"]
#complete = complete.drop(columns=["no. of commited crimes '95 ", "no. of commited crimes '96 "])
# complete

In [ ]:
sns.distplot(complete['no. of commited crimes per capita'])

In [ ]:
complete['no. of commited crimes per capita'].plot.box();

#### Remarks:
The number of crimes per capita follows the general district data distribution. One peculiarity by 'crossing data' is that the second spike in this case corresponds to places with higher numbers of inhabitants.

### 1.4.21 Card Type - Numerical Attribute

In [ ]:
complete['card type'].describe()

In [ ]:
complete['card type'].value_counts().plot(kind='bar')

#### Filling Missing Values with 'None'

In [ ]:
#complete['card type'] = complete['card type'].fillna('None')

In [ ]:
complete['card type'].value_counts()

In [ ]:
fig, ax = plt.subplots()
ax.pie(complete['card type'].value_counts(), labels=complete['card type'].unique(), autopct='%1.1f%%', explode = [0.1,0.1,0.2,0.5])
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Frequency types')

plt.show()

#### Remarks:
Missing values ('NaN') are infered to be 'no card existent' and therefore replaced by 'None' because in the 90s the popularity of the credit was still low.
Despite the very small amount of entries that have card data, this atribute can still be considered useful: for example, to have a card might lead to a higher chance to pay a loan.

### 1.4.22 Birthdate - Date Attribute

In [ ]:
complete['birthdate'].describe()

In [ ]:
print("Min: ", end="");                 print(complete['birthdate'].min())
print("Max: ", end="");                 print(complete['birthdate'].max())

In [ ]:
complete['birthdate'].value_counts()[:50].plot(kind='bar', figsize=(15,3))

#### Remarks:
The birthdate of an account owner, a priori, doesn't seem to be related at all with the possible outcome of a loan, but might be useful to detect anomalies like the possibility of an under 18 person to contract a loan which is invalid.

### 1.4.23 Client Sex - Nominal Categorical Attribute

In [ ]:
complete['sex'].describe()

In [ ]:
complete['sex'].value_counts().plot(kind='bar')

In [ ]:
fig, ax = plt.subplots()
ax.pie(complete['sex'].value_counts(), labels=complete['sex'].unique(), autopct='%1.1f%%', startangle = 90)
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Sex distribution')

plt.show()

#### Remarks:
There isn't any unbalance in the total distribution of individuals regarding their sex which should lead to a plausible atribute for loan prediction.

### 1.4.23 Account has disponent or not - Nominal Categorical Attribute

In [ ]:
complete['has disponent'].describe()

In [ ]:
complete['has disponent'].value_counts().plot(kind='bar')

In [ ]:
fig, ax = plt.subplots()
ax.pie(complete['has disponent'].value_counts(), labels=complete['has disponent'].unique(), autopct='%1.1f%%', startangle = 90)
ax.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
ax.set_title('Account has disponent')

plt.show()

#### Remarks
Despite the frequency disparity between the two possible values, another person besides the owner associated with an account may imply a higher or lower chance of loan payment.

In [ ]:
#complete.to_csv("./complete_data_comp_final_1.csv", sep=";")

In [ ]:
#complete.loc[complete['age at loan (Years)'] < 18]

## 1.5 Multivariate Data Analysis of variability or dispersion

In [ ]:
df_num = complete.select_dtypes(include = ['float64', 'int64'])
df_num.columns

### 1.5.1 Covariance Matrix

In [ ]:
# plot the heatmap
sns.heatmap(df_num.cov(), 
        xticklabels=df_num.columns,
        yticklabels=df_num.columns)

#### Remarks

### 1.5.2 Correlation Matrix

#### 1.5.2.1 Pearson (standard correlation coefficient)

In [ ]:
pearsoncorr = df_num.corr(method='pearson').round(2)
pearsoncorr

In [ ]:
fig, ax = plt.subplots(figsize=(15,10)) 
sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            linewidth=0.5,
            annot=True,
            ax=ax)

In [ ]:
complete['loan status'].corr(complete['average salary '])

In [ ]:
from scipy import stats

In [ ]:
r, p = stats.pearsonr(complete['loan status'], complete['average salary '])
print(round(r, 4))
print(round(p, 4))

In [ ]:
corr_df = pd.DataFrame(columns=['r', 'p'])

for col in complete:
    if pd.api.types.is_numeric_dtype(complete[col]) and col != 'loan status':
        r, p = stats.pearsonr(complete['loan status'], complete[col])
        corr_df.loc[col] = [round(r, 3), round(p, 3)]
    
corr_df

In [ ]:
corr_df.loc[(corr_df['p'] <= 0.05)]

#### 1.5.2.2 Spearman (Spearman rank correlation)

In [ ]:
spearmancorr = df_num.corr(method='spearman').round(2)
spearmancorr

In [ ]:
fig, ax = plt.subplots(figsize=(15,10)) 

# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(spearmancorr)

sns.heatmap(spearmancorr, 
            xticklabels=spearmancorr.columns,
            yticklabels=spearmancorr.columns,
            cmap='RdBu_r',
            linewidth=0.5,
            annot=True,
            ax=ax,
           mask=matrix)

#### Remarks

## 1.6 Data Visualization: Bivariate/Multivariate Graphs

### N Transactions / Operations / Types 

In [ ]:
#sns.pairplot(df_num[['# trans', 'oper #1', 'oper #2', 'oper #3', 'oper #4', 'oper #5', 'type #1', 'type #2', 'type #3']])

### avg trans amount / avg trans balance

In [ ]:
#complete[['avg trans amount', 'avg trans balance']].plot.scatter(x="avg trans amount", y="avg trans balance");

In [ ]:
#complete[['avg trans amount', 'avg trans balance']].plot.hexbin(x="avg trans amount", y="avg trans balance", gridsize=25);

### loan amount / avg trans amount / avg trans balance

In [ ]:
#fig, ax = plt.subplots()
#complete.plot(kind='scatter',x='avg trans amount', y='avg trans balance', c='loan amount', colormap='RdBu_r', ax=ax)
#plt.show()

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(complete['avg trans amount'], complete['avg trans balance'], complete["loan amount"], c='g', marker='o')

ax.set_xlabel('avg trans amount')
ax.set_ylabel('avg trans balance')
ax.set_zlabel('loan amount')


plt.show()

### loan duration (mths) / loan amount

In [ ]:
#complete[['loan duration (mths)', 'loan amount']].plot.hexbin(x='loan duration (mths)', y='loan amount', gridsize=25);

### loan payments / avg trans amount, avg trans balance, loan amount

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['loan payments']
y = complete['avg trans amount']
z = complete['avg trans balance']
c = complete['loan amount']

ax.set_xlabel('loan payments')
ax.set_ylabel('avg trans amount')
ax.set_zlabel('avg trans balance')

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
plt.show()

#### Remarks: loan amount increase seem to imply higher loan payments as expected

### loan status / loan amount, loan payments

In [ ]:
#fig, ax = plt.subplots()
#complete.plot(kind='scatter',x='loan amount', y='loan payments', c='loan status', colormap='winter', ax=ax)
#plt.show()

In [ ]:
#complete[['loan amount', 'loan payments', 'loan status']].plot.hexbin(x='loan amount', y='loan payments', C='loan status', gridsize=25);

### no. of cities / no. of inhabitants

In [ ]:
#complete[['no. of cities ', 'no. of inhabitants']].plot.scatter(x='no. of cities ', y='no. of inhabitants');

### ratio of urban inhabitants / no. of inhabitants, no. of cities

In [ ]:
#fig, ax = plt.subplots()
#complete.plot(kind='scatter',x='no. of cities ', y='no. of inhabitants', c='ratio of urban inhabitants ', colormap='RdBu_r', ax=ax)
#ax.set_ylabel("no. of inhabitants")
#plt.show()

### avg salary / no. of inhabitants, no. of cities, ratio of urban inhabitants

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['average salary ']
y = complete['no. of inhabitants']
z = complete['no. of cities ']
c = complete['ratio of urban inhabitants ']

ax.set_xlabel('avg salary')
ax.set_ylabel('no. of inhabitants')
ax.set_zlabel('no. of cities')

img = ax.scatter(x, y, z, c=c, cmap='RdBu_r')
fig.colorbar(img)
plt.show()

### enterpreneurs / no. of inhabitants, ratio of urban inhabitants, avg salary

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['no. of enterpreneurs per 1000 inhabitants ']
y = complete['no. of inhabitants']
z = complete['average salary ']
c = complete['ratio of urban inhabitants ']

ax.set_xlabel('no. of enterpreneurs per 1000 inhabitants')
ax.set_ylabel('no. of inhabitants')
ax.set_zlabel('average salary')

img = ax.scatter(x, y, z, c=c, cmap='RdBu_r')
fig.colorbar(img)
plt.show()

### unemploymant rate / no. of inhabitants, avg salary, enterpreneurs

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['unemploymant rate']
y = complete['no. of inhabitants']
z = complete['no. of enterpreneurs per 1000 inhabitants ']
c = complete['average salary ']

ax.set_xlabel('unemploymant rate')
ax.set_ylabel('no. of inhabitants')
ax.set_zlabel('no. of enterpreneurs per 1000 inhabitants')

img = ax.scatter(x, y, z, c=c, cmap='RdBu_r')
fig.colorbar(img)
plt.show()

### crimes / no. of cities, no. of inhabitants, ratio of urban inhabitants, avg salary, unemployment

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['ratio of urban inhabitants ']
y = complete['no. of inhabitants']
z = complete['average salary ']
c = complete['no. of commited crimes per capita']

ax.set_xlabel('ratio of urban inhabitants')
ax.set_ylabel('no. of inhabitants')
ax.set_zlabel('average salary')

img = ax.scatter(x, y, z, c=c, cmap='RdBu_r')
fig.colorbar(img)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['ratio of urban inhabitants ']
y = complete['no. of cities ']
z = complete['average salary ']
c = complete['no. of commited crimes per capita']

ax.set_xlabel('ratio of urban inhabitants')
ax.set_ylabel('no. of cities')
ax.set_zlabel('average salary')

img = ax.scatter(x, y, z, c=c, cmap='RdBu_r')
fig.colorbar(img)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = complete['ratio of urban inhabitants ']
y = complete['unemploymant rate']
z = complete['average salary ']
c = complete['no. of commited crimes per capita']

ax.set_xlabel('ratio of urban inhabitants')
ax.set_ylabel('unemploymant rate')
ax.set_zlabel('average salary')

img = ax.scatter(x, y, z, c=c, cmap='RdBu_r')
fig.colorbar(img)
plt.show()

# --- Under here not 'redone' ---

In [ ]:
temp = complete[['sex', 'loan status']]

In [ ]:
sns.swarmplot(x="loan amount", y="sex", data=complete[['sex', 'loan amount']])

In [ ]:
ax = sns.swarmplot(x="loan status", y="average salary ", hue="sex", data=complete[['average salary ', 'loan status', 'sex']])

In [ ]:
# Number of transactions, loan amount, avg amount per transaction

fig, ax = plt.subplots()
complete.plot(kind='scatter',x='# trans', y='loan amount', c='avg trans amount', colormap='viridis', ax=ax)
ax.set_xlabel("# trans")
plt.show()

In [ ]:
# %matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(complete['average salary '], complete['loan amount'], complete["unemploymant rate"], c='g', marker='o')

ax.set_xlabel('average salary')
ax.set_ylabel('loan amount')
ax.set_zlabel('unemploymant rate')


plt.show()

#### Missing Values

In [ ]:
total = complete.isnull().sum().sort_values(ascending=False)
percent = (complete.isnull().sum()/complete.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print("Total number of rows: ", end=""); print(complete.shape[0])
missing_data.head(10)

In [ ]:
#df_num = complete.select_dtypes(include = ['float64', 'int64'])
#df_num.head()

In [ ]:
#df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
corr = complete.corr()['loan status']
corr

In [ ]:
#sns.heatmap(complete.corr(method='pearson'))

#### Outliers

In [ ]:
#subset = complete[['loan amount', 'loan payments']]
#subset['loan payments'] = subset['loan payments'] * 30
#subset.plot.box();

In [ ]:
#complete['average salary '].plot.box();

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
sns.distplot(complete['loan amount'])
plt.subplot(1,2,2)
sns.distplot(complete['average salary '])
plt.show()

In [ ]:
# Finding how many 
df_mask = complete['average salary '] > 12000
# print(len(complete[df_mask].index))
complete[df_mask]